# WRI Global Power Plant Database Ingestion Pipeline (Step 4)
# Metadata ingestion in OpenMetadata using Python APIs
Copyright (C) 2021 OS-Climate

This sample shows:
* How to connect to OpenMetadata
* How to query and interact with objects such as tables via the Python API
* How to ingest additional metadata information using the REST API


Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# OpenMetadata Python APIs
%pip install openmetadata-ingestion-core
%pip install openmetadata-ingestion==0.10.3

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools

In [2]:
# Report exceptions only in cells output
%xmode Minimal

Exception reporting mode: Minimal


In [3]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import osc_ingest_trino as osc
from metadata.generated.schema.entity.services.connections.metadata.openMetadataConnection import (
    OpenMetadataConnection,
    AuthProvider
)
from metadata.generated.schema.security.client.openMetadataJWTClientConfig import OpenMetadataJWTClientConfig
from metadata.ingestion.ometa.ometa_api import OpenMetadata
from metadata.generated.schema.entity.services.databaseService import (
    DatabaseService,
    DatabaseServiceType,
    DatabaseConnection,
)

In [4]:
# Load environment variables from credentials.env
osc.load_credentials_dotenv()

Initialize OpenMetadata

In [5]:
server_config = OpenMetadataConnection(
    hostPort=os.environ['OPENMETA_ENDPOINT'],
    authProvider=AuthProvider.openmetadata,
    securityConfig=OpenMetadataJWTClientConfig(
        jwtToken=os.getenv('JWT_TOKEN', os.environ['OPENMETA_TOKEN'])
    )
)
metadata = OpenMetadata(server_config)
print(metadata.health_check())

True


In [6]:
print(metadata.client.get('/databases'))

{'data': [{'id': '6a2391a9-dd2b-409d-afdf-1674b5fdbb50', 'name': 'osc_datacommons_dev', 'fullyQualifiedName': 'OSC-CL1-trino.osc_datacommons_dev', 'version': 0.1, 'updatedAt': 1655211682414, 'updatedBy': 'ingestion-bot', 'href': 'http://openmetadata-openmetadata.apps.odh-cl2.apps.os-climate.org/api/v1/databases/6a2391a9-dd2b-409d-afdf-1674b5fdbb50', 'service': {'id': '2b84ea88-b436-4e01-b26a-f7ca6f4afdb1', 'type': 'databaseService', 'name': 'OSC-CL1-trino', 'fullyQualifiedName': 'OSC-CL1-trino', 'description': '', 'deleted': False, 'href': 'http://openmetadata-openmetadata.apps.odh-cl2.apps.os-climate.org/api/v1/services/databaseServices/2b84ea88-b436-4e01-b26a-f7ca6f4afdb1'}, 'serviceType': 'Trino', 'default': False, 'deleted': False}, {'id': '49b55c18-e7dc-4a5e-949c-aacdb8f3211e', 'name': 'osc_datacommons_dev', 'fullyQualifiedName': 'OSC-CL2.osc_datacommons_dev', 'version': 0.1, 'updatedAt': 1655303523009, 'updatedBy': 'ingestion-bot', 'href': 'http://openmetadata-openmetadata.apps.o

In [7]:
service_query = metadata.get_by_name(entity=DatabaseService, fqdn="OSC-CL2")
print(service_query)

id=Uuid(__root__=UUID('cd3bbcb9-37ab-46f6-bc66-32764e55170a')) name=EntityName(__root__='OSC-CL2') displayName=None serviceType=<DatabaseServiceType.Trino: 'Trino'> description='' connection=DatabaseConnection(config=TrinoConnection(type=<TrinoType.Trino: 'Trino'>, scheme=<TrinoScheme.trino: 'trino'>, username='admin', password=SecretStr('**********'), hostPort='trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443', catalog='osc_datacommons_dev', database=None, proxies={}, params={}, connectionOptions=ConnectionOptions(), connectionArguments=ConnectionArguments(), supportsMetadataExtraction=SupportsMetadataExtraction(__root__=True), supportsProfiler=SupportsProfiler(__root__=True))) pipelines=None version=EntityVersion(__root__=0.1) updatedAt=Timestamp(__root__=1655299738578) updatedBy='humair88' owner=None href=Href(__root__=AnyUrl('http://openmetadata-openmetadata.apps.odh-cl2.apps.os-climate.org/api/v1/services/databaseServices/cd3bbcb9-37ab-46f6-bc66-32764e55170a', scheme='h

In [8]:
print(metadata.client.get('/tables')['data'][0]['fullyQualifiedName'])

OSC-CL1-trino.osc_datacommons_dev.company_data.fundamental_data


In [9]:
my_table = metadata.get_by_name(entity=Table, fqdn="OSC-CL1-trino.osc_datacommons_dev.company_data.fundamental_data")
print(my_table)

NameError: name 'Table' is not defined